### SPRINT 4
#### TEAM ID: PNT2022TMID21442

## 1. Data Collection

In [ ]:
pwd

In [ ]:
!pip install keras
!pip install tensorflow

In [ ]:
import keras
keras.__version__

In [ ]:
import tensorflow
tensorflow.__version__

# Loading our data

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='dlfSXsHrHx9xly751DrA5r13Jmo2sWxk76JXeII4pi2N',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'classificationofarrhythmiabyusing-donotdelete-pr-7j5j6vzxaehedv'
object_key = 'data.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
print(file_paths)
for path in file_paths:
    unzip.extract(path)

In [ ]:
pwd

## 2. Image Preprocessing

#### Import The Image data Generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#setting parameter for Image bata agumentation to the traing data
train_datagen= ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

#Image Data agumentation to the testing data
test_datagen = ImageDataGenerator(rescale=1./255)

#### Apply Image Data Generator Functionality To Trainset and Testset

In [ ]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(directory = "/home/wsuser/work/data/train", target_size = (64, 64), batch_size = 32, class_mode='categorical')

#performing data agumentation to test data
x_test =  test_datagen.flow_from_directory(directory = "/home/wsuser/work/data/test" , target_size = (64, 64), batch_size = 32, class_mode="categorical")

#### Confiqure Image Data Generator Class

In [ ]:
x_train.class_indices

# **Model Buliding**

Adding Layers:

In [ ]:
#Import req. Lib.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
# Build a CNN Block:
model = Sequential() #intializing sequential model
model.add(Convolution2D(32,(3,3),activation='relu', input_shape=(64,64,3))) #convolution laye
model.add(MaxPooling2D(pool_size=(2, 2))) #Maxpooling layer
model.add(Flatten()) #Flatten layer
model.add(Dense(400,activation='relu')) #Hidden Layer 1
model.add(Dense(200,activation='relu')) #Hidden Layer 2
model.add(Dense(6,activation='softmax')) #Output Layer

In [ ]:
model.summary()#summary of our model

Compiling :

In [ ]:
# Compiling The Model...
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


Fit / Train The Model :

In [ ]:
#Train Model:
model.fit_generator(x_train,
steps_per_epoch=len(x_train),
epochs=10,
validation_data=x_test,
validation_steps=len(x_test))

Saving The Model :

In [ ]:
model.save('ECG.h5')


In [ ]:
!tar -zcvf ECG-Image-based-heartbeat-classification-model_new.tgz ECG.h5

In [ ]:
ls -1

In [ ]:
!pip install watson-machine-learning-client --upgrade

In [ ]:
# Replace the credentials that you got from watson machine learning service
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"TkHXR1FH4Bk9bPi9BRyxTUiUQrPT9AAffkQ-ND1CUuSS"
                  }
client = APIClient(wml_credentials)


In [ ]:
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, 'image_classification')
print("Space UID = "+ space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list(limit=100)

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='ECG-Image-based-heartbeat-classification-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"Image_Classification",
client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})

model_id = client.repository.get_model_uid(model_details)
    

In [ ]:
client.repository.download(model_id, 'my_model.tar.gz')

In [ ]:
!tar xvzf my_model.tar.gz

In [ ]:
!ls -1

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model = load_model("ECG.h5")
